In [1]:
import numpy as np
from deck import full_euchre_deck
from branch_calc import (
    filter_branch_by_hand,
    setdiff2d_idx,
    suit_id,
    find_winner,
    calc_all_possible_hands,
)
from dealer import dealer

In [2]:
game = dealer(deck=full_euchre_deck ,players=4)
game.deal_cards()
hands = [game.hand1, game.hand2, game.hand3, game.hand4]
first_play = [game.hand1[0], game.hand1[1], game.hand1[2], game.hand1[3], game.hand1[4]]

In [3]:
# all playable hands
all_possible_tricks = calc_all_possible_hands(hands=hands)
len(all_possible_tricks)

625

In [4]:
branches = {}
i = 0
# creates 5 branches with 125 possible tricks each
for card in first_play:
    i += 1
    contains_target = np.any(np.all(all_possible_tricks == card, axis=-1), axis=1)
    branches[f'branch{i}'] = all_possible_tricks[contains_target]

In [5]:
# creates all possible tricks based on rules for following suit
i=-1
for branch in branches.keys():
    i+=1
    branches[branch] = filter_branch_by_hand(branches[branch] , game.hand2, 1, first_play[i])
    branches[branch] = filter_branch_by_hand(branches[branch], game.hand3, 2, first_play[i])
    branches[branch] = filter_branch_by_hand(branches[branch], game.hand4, 3, first_play[i])

In [ ]:
# 2nd round progression. For each trick winner, 4 different cards can be played the following round
# so if round one of a branch results in 5 possible tricks, the branch would continue with 20 possible tricks
# the following round because each lead player has 4 options. 

# this is getting very complicated
i=0
for trick in branches['branch1']:
    rd2_lead = find_winner(lead=trick[0], trick=trick)
    print(rd2_lead)
    hands = [setdiff2d_idx(game.hand1, trick),
    setdiff2d_idx(game.hand2, trick),
    setdiff2d_idx(game.hand3, trick),
    setdiff2d_idx(game.hand4, trick)]
    round2 = calc_all_possible_hands(hands=hands)
    for card in hands[rd2_lead]:
        i += 1
        contains_target = np.any(np.all(round2 == card, axis=-1), axis=1)
        branches[f'branch1_{i}'] = round2[contains_target]
        print(card)


3
[-12   0]
[  0 100]
[  0 120]
[-9  0]
2
[  0 -12]
[-14   0]
[   0 -135]
[  0 130]
3
[ 0 90]
[-12   0]
[  0 120]
[-9  0]
3
[ 0 90]
[-12   0]
[  0 100]
[-9  0]
2
[  0 -12]
[-14   0]
[   0 -135]
[  0 130]


In [ ]:
# gotta fix this block to account for different players starting the game. 

for j in np.arange(1, 4*len(branches['branch1'])+1):
    branches[f'branch1_{j}'] = filter_branch_by_hand(branches[f'branch1_{j}'] , game.hand2, 1, card)
    branches[f'branch1_{j}'] = filter_branch_by_hand(branches[f'branch1_{j}'], game.hand3, 2, card)
    branches[f'branch1_{j}'] = filter_branch_by_hand(branches[f'branch1_{j}'], game.hand4, 3, card)

In [12]:
branches.keys()

dict_keys(['branch1', 'branch2', 'branch3', 'branch4', 'branch5', 'branch1_1', 'branch1_2', 'branch1_3', 'branch1_4', 'branch1_5', 'branch1_6', 'branch1_7', 'branch1_8', 'branch1_9', 'branch1_10', 'branch1_11', 'branch1_12', 'branch1_13', 'branch1_14', 'branch1_15', 'branch1_16', 'branch1_17', 'branch1_18', 'branch1_19', 'branch1_20'])

In [9]:
len(branches['branch1_1'])

64

In [10]:
# in order to move on from here we need to figure out scoring each trick.
# when we score each trick we can figure out which hand leads round 2, and maybe rinse 
# and repeat the process above. 

In [11]:
# len(second_round.keys()), second_round